# Build an End-to-End System

This puts together the chain of prompts that you saw throughout the course.

## Setup
#### Load the API key and relevant Python libaries.
In this course, we've provided some code that loads the OpenAI API key for you.

In [3]:
import os
import openai
import sys
sys.path.append('../..')
import utils

import panel as pn  # GUI
pn.extension()

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [4]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message["content"]

### Some functions that are not mentioned in the main notebook as it uses a pre-defined environment

In [31]:
from collections import defaultdict
import json

def find_category_and_product_only(user_input,products_and_category):
    delimiter = "####"
    system_message = f"""
    You will be provided with customer service queries. \
    The customer service query will be delimited with {delimiter} characters.
    Output a python list of objects, where each object has the following format:
    'category': <one of Computers and Laptops, Smartphones and Accessories, Televisions and Home Theater Systems, \
    Gaming Consoles and Accessories, Audio Equipment, Cameras and Camcorders>,
    OR
    'products': <a list of products that must be found in the allowed products below>

    Where the categories and products must be found in the customer service query.
    If a product is mentioned, it must be associated with the correct category in the allowed products list below.
    If no products or categories are found, output an empty list.

    Allowed products: 
    Computers and Laptops category:
    TechPro Ultrabook
    BlueWave Gaming Laptop
    PowerLite Convertible
    TechPro Desktop
    BlueWave Chromebook

    Smartphones and Accessories category:
    SmartX ProPhone
    MobiTech PowerCase
    SmartX MiniPhone
    MobiTech Wireless Charger
    SmartX EarBuds

    Televisions and Home Theater Systems category:
    CineView 4K TV
    SoundMax Home Theater
    CineView 8K TV
    SoundMax Soundbar
    CineView OLED TV

    Gaming Consoles and Accessories category:
    GameSphere X
    ProGamer Controller
    GameSphere Y
    ProGamer Racing Wheel
    GameSphere VR Headset

    Audio Equipment category:
    AudioPhonic Noise-Canceling Headphones
    WaveSound Bluetooth Speaker
    AudioPhonic True Wireless Earbuds
    WaveSound Soundbar
    AudioPhonic Turntable

    Cameras and Camcorders category:
    FotoSnap DSLR Camera
    ActionCam 4K
    FotoSnap Mirrorless Camera
    ZoomMaster Camcorder
    FotoSnap Instant Camera

        Only output the list of objects, nothing else.
    """
    messages =  [  
    {'role':'system', 'content': system_message},    
    {'role':'user', 'content': f"{delimiter}{user_input}{delimiter}"},  
    ] 
    return get_completion_from_messages(messages)

def get_products():
    with open("products_file", 'r') as file:
        products = json.load(file)
    return products

def get_products_and_category():
    """
    Used in L5
    """
    products = get_products()
    products_by_category = defaultdict(list)
    for product_name, product_info in products.items():
        category = product_info.get('category')
        if category:
            products_by_category[category].append(product_info.get('name'))
    
    return dict(products_by_category)

def read_string_to_list(input_string):
    if input_string is None:
        return None

    try:
        input_string = input_string.replace("'", "\"")  # Replace single quotes with double quotes for valid JSON
        data = json.loads(input_string)
        return data
    except json.JSONDecodeError:
        print("Error: Invalid JSON string")
        return None
    
def get_product_by_name(name):
    products = get_products()
    return products.get(name, None)
    
def generate_output_string(data_list):
    output_string = ""

    if data_list is None:
        return output_string

    for data in data_list:
        try:
            if "products" in data:
                products_list = data["products"]
                for product_name in products_list:
                    product = get_product_by_name(product_name)
                    if product:
                        output_string += json.dumps(product, indent=4) + "\n"
                    else:
                        print(f"Error: Product '{product_name}' not found")
            elif "category" in data:
                category_name = data["category"]
                category_products = get_products_by_category(category_name)
                for product in category_products:
                    output_string += json.dumps(product, indent=4) + "\n"
            else:
                print("Error: Invalid object format")
        except Exception as e:
            print(f"Error: {e}")

    return output_string

## System of chained prompts for processing the user query

Note - I have made some changes in the code to call the above functions directly and not via `utils`

In [34]:
def process_user_message(user_input, all_messages, debug=True):
    delimiter = "```"
    
    # Step 1: Check input to see if it flags the Moderation API or is a prompt injection
    response = openai.Moderation.create(input=user_input)
    moderation_output = response["results"][0]

    if moderation_output["flagged"]:
        print("Step 1: Input flagged by Moderation API.")
        return "Sorry, we cannot process this request."

    if debug: print("Step 1: Input passed moderation check.")
    
    category_and_product_response = find_category_and_product_only(user_input, get_products_and_category())
    #print(print(category_and_product_response)
    # Step 2: Extract the list of products
    category_and_product_list = read_string_to_list(category_and_product_response)
    #print(category_and_product_list)

    if debug: print("Step 2: Extracted list of products.")

    # Step 3: If products are found, look them up
    product_information = generate_output_string(category_and_product_list)
    if debug: print("Step 3: Looked up product information.")

    # Step 4: Answer the user question
    system_message = f"""
    You are a customer service assistant for a large electronic store. \
    Respond in a friendly and helpful tone, with concise answers. \
    Make sure to ask the user relevant follow-up questions.
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': f"{delimiter}{user_input}{delimiter}"},
        {'role': 'assistant', 'content': f"Relevant product information:\n{product_information}"}
    ]

    final_response = get_completion_from_messages(all_messages + messages)
    if debug:print("Step 4: Generated response to user question.")
    all_messages = all_messages + messages[1:]

    # Step 5: Put the answer through the Moderation API
    response = openai.Moderation.create(input=final_response)
    moderation_output = response["results"][0]

    if moderation_output["flagged"]:
        if debug: print("Step 5: Response flagged by Moderation API.")
        return "Sorry, we cannot provide this information."

    if debug: print("Step 5: Response passed moderation check.")

    # Step 6: Ask the model if the response answers the initial user query well
    user_message = f"""
    Customer message: {delimiter}{user_input}{delimiter}
    Agent response: {delimiter}{final_response}{delimiter}

    Does the response sufficiently answer the question?
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]
    evaluation_response = get_completion_from_messages(messages)
    if debug: print("Step 6: Model evaluated the response.")

    # Step 7: If yes, use this answer; if not, say that you will connect the user to a human
    if "Y" in evaluation_response:  # Using "in" instead of "==" to be safer for model output variation (e.g., "Y." or "Yes")
        if debug: print("Step 7: Model approved the response.")
        return final_response, all_messages
    else:
        if debug: print("Step 7: Model disapproved the response.")
        neg_str = "I'm unable to provide the information you're looking for. I'll connect you with a human representative for further assistance."
        return neg_str, all_messages


In [35]:
user_input = "tell me about the smartx pro phone and the fotosnap camera, the dslr one. Also what tell me about your tvs"
response,_ = process_user_message(user_input,[])
print(response)

Step 1: Input passed moderation check.
Error: Invalid JSON string
Step 2: Extracted list of products.
Step 3: Looked up product information.
Step 4: Generated response to user question.
Step 5: Response passed moderation check.
Step 6: Model evaluated the response.
Step 7: Model approved the response.
Sure, I'd be happy to help! 

The SmartX Pro phone is a high-end smartphone with a large, high-resolution display, powerful processor, and advanced camera features. It's a great choice for anyone who wants a top-of-the-line device with all the latest features.

The Fotosnap DSLR camera is a professional-grade camera that's perfect for serious photographers. It has a large sensor, interchangeable lenses, and advanced manual controls, allowing you to capture stunning photos with incredible detail and clarity.

As for our TVs, we have a wide range of options to choose from, including LED, OLED, and QLED models in a variety of sizes and resolutions. Whether you're looking for a basic TV for y

### Function that collects user and assistant messages over time

In [36]:
def collect_messages(debug=False):
    user_input = inp.value_input
    if debug: print(f"User Input = {user_input}")
    if user_input == "":
        return
    inp.value = ''
    global context
    #response, context = process_user_message(user_input, context, utils.get_products_and_category(),debug=True)
    response, context = process_user_message(user_input, context, debug=False)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(user_input, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

### Chat with the chatbot!
Note that the system message includes detailed instructions about what the OrderBot should do.

In [37]:
panels = [] # collect display 

context = [ {'role':'system', 'content':"You are Service Assistant"} ]  

inp = pn.widgets.TextInput( placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Service Assistant")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Traceback (most recent call last):
 File "d:\codes\dl\building-systems-with-the-chatgpt-api\venv\lib\site-packages\pyviz_comms\__init__.py", line 340, in _handle_msg
 self._on_msg(msg)
 File "d:\codes\dl\building-systems-with-the-chatgpt-api\venv\lib\site-packages\panel\viewable.py", line 465, in _on_msg
 patch.apply_to_document(doc, comm.id if comm else None)
 File "d:\codes\dl\building-systems-with-the-chatgpt-api\venv\lib\site-packages\bokeh\protocol\messages\patch_doc.py", line 104, in apply_to_document
 invoke_with_curdoc(doc, lambda: doc.apply_json_patch(self.payload, setter=setter))
 File "d:\codes\dl\building-systems-with-the-chatgpt-api\venv\lib\site-packages\bokeh\document\callbacks.py", line 443, in invoke_with_curdoc
 return f()
 File "d:\codes\dl\building-systems-with-the-chatgpt-api\venv\lib\site-packages\bokeh\protocol\messages\patch_doc.py", line 104, in <lambda>
 invoke_with_curdoc(doc, lambda: doc.apply_json_patch(self.payload, setter=setter))
 File "d:\codes\dl\building-systems-with-the-chatgpt-api\venv\lib\site-packages\bokeh\document\document.py", line 376, in apply_json_patch
 DocumentPatchedEvent.handle_event(self, event, setter)
 File "d:\codes\dl\building-systems-with-the-chatgpt-api\venv\lib\site-packages\bokeh\document\events.py", line 246, in handle_event
 event_cls._handle_event(doc, event)
 File "d:\codes\dl\building-systems-with-the-chatgpt-api\venv\lib\site-packages\bokeh\document\events.py", line 281, in _handle_event
 cb(event.msg_data)
 File "d:\codes\dl\building-systems-with-the-chatgpt-api\venv\lib\site-packages\bokeh\document\callbacks.py", line 390, in trigger_event
 model._trigger_event(event)
 File "d:\codes\dl\building-systems-with-the-chatgpt-api\venv\lib\site-packages\bokeh\util\callback_manager.py", line 113, in _trigger_event
 self.document.callbacks.notify_event(cast(Model, self), event, invoke)
 File "d:\codes\dl\building-systems-with-the-chatgpt-api\venv\lib\site-packages\bokeh\document\callbacks.py", line 260, in notify_event
 invoke_with_curdoc(doc, callback_invoker)
 File "d:\codes\dl\building-systems-with-the-chatgpt-api\venv\lib\site-packages\bokeh\document\callbacks.py", line 443, in invoke_with_curdoc
 return f()
 File "d:\codes\dl\building-systems-with-the-chatgpt-api\venv\lib\site-packages\bokeh\util\callback_manager.py", line 109, in invoke
 cast(EventCallbackWithEvent, callback)(event)
 File "d:\codes\dl\building-systems-with-the-chatgpt-api\venv\lib\site-packages\panel\reactive.py", line 479, in _comm_event
 state._handle_exception(e)
 File "d:\codes\dl\building-systems-with-the-chatgpt-api\venv\lib\site-packages\panel\io\state.py", line 432, in _handle_exception
 raise exception
 File "d:\codes\dl\building-systems-with-the-chatgpt-api\venv\lib\site-packages\panel\reactive.py", line 477, in _comm_event
 self._process_bokeh_event(doc, event)
 File "d:\codes\dl\building-systems-with-the-chatgpt-api\venv\lib\site-packages\panel\reactive.py", line 414, in _process_bokeh_event
 self._process_event(event)
 File "d:\codes\dl\building-systems-with-the-chatgpt-api\venv\lib\site-packages\panel\widgets\button.py", line 242, in _process_event
 self.param.trigger('value')
 File "d:\codes\dl\building-systems-with-the-chatgpt-api\venv\lib\site-packages\param\parameterized.py", line 1993, in trigger
 self_.set_param(**dict(params, **triggers))
 File "d:\codes\dl\building-systems-with-the-chatgpt-api\venv\lib\site-packages\param\parameterized.py", line 1929, in set_param
 return self_.update(kwargs)
 File "d:\codes\dl\building-systems-with-the-chatgpt-api\venv\lib\site-packages\param\parameterized.py", line 1902, in update
 self_._batch_call_watchers()
 File "d:\codes\dl\building-systems-with-the-chatgpt-api\venv\lib\site-packages\param\parameterized.py", line 2063, in _batch_call_watchers
 self_._execute_watcher(watcher, events)
 File "d:\codes\dl\building-systems-with-the-chatgpt-api\venv\lib\site-packages\param\parameterized.py", line 2025, in _execute_watcher
 watcher.fn(*args, *

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Service Assistant')
    [2] ParamFunction(function, _pane=Str, height=300, loading_indicator=True)